In [3]:
import requests, zipfile, io
import pandas as pd

# 1) Acquisition des données

## Récupération des données socio-économiques 

Disponibles sur le site de l'INSEE.

In [6]:
url = 'https://www.insee.fr/fr/statistiques/fichier/5359146/dossier_complet.zip'

In [7]:
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("INSEE/")

## Récupération des données des élections municipales de 2020

Disponibles sur le site data.gouv. Méthode choisie à cause de problèmes d'encodage des caractères accentués.

In [8]:
!curl -O https://static.data.gouv.fr/resources/elections-municipales-2020-resultats/20200525-123727/2020-05-18-resultats-par-niveau-burvot-t1-france-entiere.xlsx

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 34.7M  100 34.7M    0     0  14.8M      0  0:00:02  0:00:02 --:--:-- 14.8M


In [52]:
# Attention, l'éxécution de cette cellule peut prendre un certain temps
municipales_2020 = pd.read_excel('2020-05-18-resultats-par-niveau-burvot-t1-france-entiere.xlsx')

## Récupération des données des élections municipales de 2014

Il s'agit de la seule élection où une liste finale des maires élus existe.

In [68]:
municipales_2014 = pd.read_excel('https://www.data.gouv.fr/storage/f/2014-06-17T12-56-58/maires-17-06-2014.xlsx')

# 2) Nettoyage des données

## Elections municipales de 2014:

In [69]:
#Renommer les colonnes (attention code non idem-potent)
municipales_2014.columns = municipales_2014.iloc[1]
municipales_2014.drop(labels=[0,1], axis=0, inplace=True)
municipales_2014.reset_index(drop=True, inplace=True)

# 3) Détermination des maires

## Elections municipales de 2020:

In [186]:
def compte_candidats(liste):
    return len(liste) - sum(pd.isna(liste).astype(int))

In [187]:
communes = municipales_2020['Libellé de la commune'].unique()

noms = [f'Nom.{i}' for i in range(1,62)]
noms.insert(0, 'Nom')

voix = [f'Voix.{i}' for i in range(1,62)]
voix.insert(0, 'Voix')

In [242]:
maires = pd.DataFrame(columns=['Villes', 'Maire'])

for i in communes:
    nombre_candidats = compte_candidats(list(municipales_2020.loc[municipales_2020['Libellé de la commune']==i][noms].iloc[0]))   
    max_voix = 0
    indice_max = 0
    if i=="Saint-Savin":
        print(i)
    for j in range(nombre_candidats):
        actuel = municipales_2020.loc[municipales_2020['Libellé de la commune']==i][voix[j]].sum()
        if actuel > max_voix:
            max_voix = actuel
            indice_max = j
    maires = maires.append([i, municipales_2020.loc[municipales_2020['Libellé de la commune']==i][noms[indice_max]]])

KeyboardInterrupt: 

On observe une singularité pour les communes de moins de 1 000 habitants. Pour ces communes, l'élection des conseillers municipaux se déroule au scrutin majoritaire plurinominal à deux tours, avec panachage. La somme des voix de tous les candidats peut donc être supérieure au nombre de votants.

Nous allons donc ne pas pouvoir analyser ces communes.